# Utils

In [ ]:
import numpy as np

In [ ]:
##### helper functions

# function to find corrept element in result dict of prediction based on predicate
def get_predicate_arguments(predicate, prediction):

    predicate_arguments = None
    verbs = []
    
    for i in range(len(prediction['verbs'])):
        verb = prediction['verbs'][i]['verb']
        verbs.append(verb)
        #print('-', verb == predicate, ' || verb: ',  verb, '- predicate: ', predicate)
        if verb == predicate:
            predicate_arguments = prediction['verbs'][i]
            break
    
    if predicate_arguments == None:
        #raise Exception(f' - Predicate not identified: predicate \"{predicate}\" not found in verbs: {verbs}') 
        predicate_arguments = {}
        #predicate_arguments['verbs'] = {}
        predicate_arguments['description'] = ''
        predicate_arguments['tags'] = []
        
    #print('# predicate_arguments ', predicate_arguments)
    return predicate_arguments


# 
'''
#def predict_srl(data):#, predictor=srl_predictor):
def predict_srl(data, predictor):
    print(type(data), data)
    pred = []
    for d in data:
        pred.append(predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)
'''

def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

#predict_and_conf = PredictorWrapper.wrap_predict(predict_srl) 



def wrap_predict(data, predictor):
    predictor = predictor
    def pred_and_conf(data):
        #preds = predict_fn(inputs)
        preds = []
        for d in data:
            preds.append(predictor.predict(d))
        confs = np.ones(len(preds))
        return preds, confs
    return pred_and_conf

#predict_and_conf = wrap_predict(predict_srl) 



# Helper function to extract target argument
#def get_arg(pred, meta, predicate, arg_target='ARG1'):
def get_arg(pred, meta, predicate, arg_target):

    
    predicate_arguments = get_predicate_arguments(predicate, pred)
            
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            #arg = t.split('-')[1]
            arg = t[t.find('-')+1:]
            if 'ARG' in arg and not arg.startswith('ARG'):
                arg = arg[arg.find('-')+1:]
                
        #print('  - token comp: ', w, arg,  arg_target, arg == arg_target)
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    
    return arg_set


# Helper function to display failures
def format_srl(x, pred, conf, label, meta):
    
    results = []
    
    try:
        predicate = meta[label['predicate']]
    except:
        predicate = label['predicate']
     
    predicate_arguments = get_predicate_arguments(predicate, pred)
    
    try:
        predicate_structure = predicate_arguments['description']
    except:
        predicate_structure = ''
        
    return predicate_structure


#
def compare_one_arg(x, pred, conf,label=None, meta=None):
    
    target = set([meta[label['target']]])
    
    try:
        predicate = meta[label['predicate']]
    except:
        predicate = label['predicate']
        
    #print('\n new comparison \n # predicate/arg_target:', predicate, label['arg_target'])
    arg = get_arg(pred, meta, predicate, arg_target=label['arg_target'])
    #print('# arg', arg)

    #print(f'# finalComparison {arg} == {target}', type(target), target in arg, target.issubset(arg))
    if target.issubset(arg):
        pass_ = True
    else:
        pass_ = False
    return pass_


one_arg_expected = Expect.single(compare_one_arg)



In [ ]:
# function to retrieve correct span from input predicate and term to search for
def get_span(term, predicate, prediction):
    
    # in case no description is available when no predicate was found
    try:
        predicate_descriptions = get_predicate_arguments(predicate, prediction)['description']
    except:
        predicate_descriptions = ''
        
    elems = re.findall(r'\[(.*?)\]', predicate_descriptions)
    
    arg  = ''
    span = ''
    
    for elem in elems:
        
        if term in elem:
            try:
                arg  = elem.split(':')[0]
                span = elem.split(':')[1].lstrip()
                break
            except ValueError:
                print(f'{elem} cannot be split') 
    return arg, span
    

# returns to lists (failures and correct) containing elems 
# each elem with sentence, span, expected, and predicted arg

def create_output_elems(test):
    
    failures = []
    correct  = []

    for sentence_id in range(len(test.data)):


        # create elem
        elem      = {}
        
        term       = test.meta[sentence_id][test.labels[sentence_id]['target']]
        predicate  = test.meta[sentence_id][test.labels[sentence_id]['predicate']]
        prediction = test.results['preds'][sentence_id]
        
        #print(' - \tpredicate', predicate, '\tterm', term, '\n \tpreds', prediction)
        arg_and_span = get_span(term, predicate, prediction)

        elem['sentence']  = test.data[sentence_id]
        #elem['span']      = arg_and_span[1]
        elem['expected']  = test.labels[sentence_id]['arg_target']
        elem['predicted'] = arg_and_span[0]
        elem['token']     = term


        # of prediction failed, store in fails
        # else store in correc
        if sentence_id in test.fail_idxs():
            failures.append(elem)
        else:
            correct.append(elem)

    return failures, correct



In [2]:
#function to create result dict
def create_result_dict():
    
    results = {}
    results['capabilities'] = {}
    
    return results


# function to append result dict
def append_result_dict_by_test(result_dict, test, testtype):

    found = False
    
    if len(result_dict['capabilities']) > 0:
        #print('if')
        for cap in result_dict['capabilities']:
            # if capability already exists
            if result_dict['capabilities'][cap]['capability_name'] == test.capability:
                #print('if_if')
                t = result_dict['capabilities'][cap]#['capability_name']
                found = True
                pass
        
    if found == False:
        #print('else')
        nr_of_capabilities = len(result_dict['capabilities'])
        result_dict['capabilities']['capability'+str(nr_of_capabilities+1)] = {}
        result_dict['capabilities']['capability'+str(nr_of_capabilities+1)]['capability_name'] = test.capability
        t = result_dict['capabilities']['capability'+str(nr_of_capabilities+1)]
        
        t['tests'] = {}


    #print('\n\n #### Appending\n - test name', test.name, '\n - dict: ', t, '\n___\n\n')
    t['tests'][test.name] = {}
    
    # storing more information if required
    #t[test.name]['summary']   = test.summary()
    #t[test.name]['fail_idxs'] = [test.fail_idxs()]
    #t[test.name]['labels']    = test.labels
    #t[test.name]['meta']      = test.meta
    #t[test.name]['results']   = test.results
    t['tests'][test.name]['data']      = test.data
    t['tests'][test.name]['testtype']  = testtype    

    failures, correct = create_output_elems(test)

    t['tests'][test.name]['failures'] = failures
    t['tests'][test.name]['correct']  = correct
    t['tests'][test.name]['failure_rate'] = len(failures)/(len(failures)+len(correct))
    

    return result_dict


# function to write out result dict
def write_out_result_dict(result_dict, path='data/checklist_results.json'):
    
    json_string = json.dumps(result_dict, indent=4)
    with open(path, 'w') as outfile:
        outfile.write(json_string)




In [ ]:
# function to generate test instances 
def create_tests(test_material_dict, nr_of_samples=100):
   
    tests = []
    
    for test in test_material_dict['tests'].keys():
        
        t = test_material_dict['tests'][test]
        
        template = editor.template(t['sentence'], 
                                   meta=True, 
                                   nsamples=nr_of_samples, 
                                   labels = {'target':     t['target'], 
                                             'arg_target': t['arg_target'], 
                                             'predicate':  t['predicate']},
                                   remove_duplicates=True)
        
        test = MFT(**template, 
                   expect     = test_material_dict['expected_fn'], 
                   capability = test_material_dict['capability'],
                   name       = test)
        
        tests.append(test)
    
    return tests


# function to wrap running and storing of tests
def run_tests(result_dict, tests, testtype, predict_and_conf):
    
    for test in tests:
        test.run(predict_and_conf)
        test.summary(format_example_fn=format_srl)
        
        result_dict = append_result_dict_by_test(result_dict, test, testtype)
        
        
    return result_dict
